# Constrained Food Recommender

In this assignment, you will implement both Content Based and Collaborative Filtering Recommenders and backtracking search (or local search) on your own

100% finished homework should contain EDA, Item and User profiles generation, Content-Based Recommender, Collaborative Filtering Recommender, and soluton to CSP problem of assigning recommendations to brekfast, lunch and dinner.

## Data loading

You will work with subset of [Academic Yelp Dataset](https://www.kaggle.com/yelp-dataset/yelp-dataset) containing list of restaurants in **yelp_business.csv** and reviews of the users in **yelp_reviews.parquet**

In [4]:
import pandas as pd
import numpy as np

df_yelp_business = pd.read_csv("yelp_business.csv").drop(columns=["Unnamed: 0"])
df_yelp_reviews = pd.read_parquet("yelp_reviews.parquet")

# Leave only users with at least 3 reviews
users_count = df_yelp_reviews.groupby("user_id").count()[["business_id"]] 
users_to_use = users_count[users_count["business_id"] > 2]
df_yelp_reviews = df_yelp_reviews[df_yelp_reviews["user_id"].isin(users_to_use.index)]

import warnings
warnings.filterwarnings("ignore")

## Exploratory data analysis

Here you will explore the data to find out what is the distribution of business categories, hours, places, user reviews, etc.

This step is needed to proceed later with item and user profiling and to clean your data if there are duplicates (e.g. duplicated reviews, the same businesses under different ids, categories tags which are highly correlated) or some artifacts not related to the main task.

(5 points)

In [5]:
df_yelp_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,404 E Green St,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...",pQeaRpvuhoEqudo3uymHIQ,"Ethnic Food, Food Trucks, Specialty Food, Impo...",Champaign,"{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",1,40.110446,-88.233073,The Empanadas House,61820,5,4.5,IL
1,4508 E Independence Blvd,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...",CsLQLiRoafpJPJSkNX2h5Q,"Food, Restaurants, Grocery, Middle Eastern",Charlotte,NaN,0,35.194894,-80.767442,Middle East Deli,28205,5,3.0,NC
2,300 John Street,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",lu7vtrp_bE9PnxWfA8g4Pg,"Japanese, Fast Food, Food Court, Restaurants",Thornhill,NaN,1,43.820492,-79.398466,Banzai Sushi,L3T 5W4,7,4.5,ON
3,"4550 East Cactus Rd, #KSFC-4","{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",vjTVxnsQEZ34XjYNS-XUpA,"Food, Pretzels, Bakeries, Fast Food, Restaurants",Phoenix,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,33.602822,-111.983533,Wetzel's Pretzels,85032,10,4.0,AZ
4,9595 W Tropicana Ave,"{'Alcohol': ""u'none'"", 'WiFi': ""u'no'"", 'GoodF...",fnZrZlqW1Z8iWgTVDfv_MA,"Mexican, Restaurants, Fast Food",Las Vegas,NaN,0,36.099738,-115.301568,Carl's Jr,89147,15,2.5,NV


In [6]:
df_yelp_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
4,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0,6TdNDKywdbjoTkizeMce8A,4,happy day finally canes near casa yes others g...,0,UgMW8bLE0QMJDCkQ1Ax5Mg
6,Pthe4qk5xh4n-ef-9bvMSg,0,2015-11-05 23:11:05,0,ZayJ1zWyWgY9S_TRLT_y9Q,5,really good place simple decor amazing food gr...,1,aq_ZxGHiri48TUXJlpRkCQ
9,Ws8V970-mQt2X9CwCuT5zw,1,2009-10-13 04:16:41,0,z4BCgTkfNtCu4XY5Lp97ww,4,twice nice laid back tried weekend southern me...,3,jOERvhmK6_lo_XGUBPws_w
16,d4qwVw4PcN-_2mK2o1Ro1g,0,2015-02-02 06:28:00,0,bVTjZgRNq8ToxzvtiVrqMA,1,10pm super bowl sunday already closed weak won...,0,2hRe26HSCAWbFRn5WChK-Q
22,9Jo1pu0y2zU6ktiwQm6gNA,20,2016-12-04 03:15:21,19,sgTnHfeaEvyOoWX4TCgkuQ,4,coconut fish cafe fantastic five stars fish ca...,24,A0j21z2Q1HGic7jW6e9h7A


In [7]:
df_yelp_business.shape[0], df_yelp_business.drop_duplicates().shape[0]

(5748, 5748)

So there are no exact duplicates. Let's also check by name.

In [8]:
df_yelp_business.name.value_counts()

McDonald's                          89
Subway Restaurants                  53
Pizza Hut                           35
Burger King                         32
Wendy's                             30
                                    ..
Alborz Restaurant                    1
Orca  The Sandwich Shoppe            1
Omni Eatery                          1
Paramount Middle Eastern Kitchen     1
Heidi's Brooklyn Deli                1
Name: name, Length: 4716, dtype: int64

It makes sense that we need to include some location markers to figure out if there are any exact matches.

In [9]:
df_yelp_business.duplicated(subset=['name', 'longitude', 'latitude']).value_counts()

False    5748
dtype: int64

Okay, so there are no same businesses under different id's.

How many categories are there?

In [10]:
category_counts = {}
for category_set in df_yelp_business.categories:
    for cat in category_set.split(', '):
        category_counts[cat] = category_counts.get(cat, 0) + 1

category_counts = pd.Series(category_counts.values(), index=category_counts.keys())
print(category_counts.shape[0])
category_counts.value_counts().sort_values(ascending=False)

411


1      130
2       39
3       26
6       19
4       16
      ... 
98       1
93       1
267      1
685      1
23       1
Length: 100, dtype: int64

There are 130 categories that only occur once, thus using them for calculating similarity is useless. If we remove them and others up to 4 occurences,
the other 200 categories can be used with a one-hot encoding.

In [11]:
category_counts = category_counts[category_counts > 4]

## Building recommender

First of all you should process user reviews to get the utility matrix containing ratings for users and businesses. There will be a lot of 0 in this matrix and it is better to store such matrices in the specialized data structure for sparce matrices. However, your working dataset is relatively small and we can use simple **pd.DataFrame** to proceed

In [12]:
def create_utility_matrix(reviews: pd.DataFrame, business: pd.DataFrame) -> pd.DataFrame:
    business_ids = business["business_id"].unique()
    users = reviews["user_id"].unique()
    ut_matrix = pd.DataFrame(0, columns=business_ids, index=users)
    for _, review in reviews.iterrows():
        ut_matrix.loc[review["user_id"], review["business_id"]] = review["stars"]
    # TODO: Rating normalization to (-1, 1) range (5 points)
    ut_matrix[ut_matrix == 0] = 3
    ut_matrix = (ut_matrix - 3) / 2
    return ut_matrix

df_utility_matrix = create_utility_matrix(df_yelp_reviews, df_yelp_business)
df_utility_matrix.head()

,pQeaRpvuhoEqudo3uymHIQ,CsLQLiRoafpJPJSkNX2h5Q,lu7vtrp_bE9PnxWfA8g4Pg,vjTVxnsQEZ34XjYNS-XUpA,fnZrZlqW1Z8iWgTVDfv_MA,rVBPQdeayMYht4Uv_FOLHg,fhNf_sg-XzZ3e7HEVGuOZg,LoRef3ChgZKbxUio-sHgQg,Ga2Bt7xfqoggTypWD5VpoQ,xFc50drSPxXkcLvX5ygqrg,...,2SfSzEd3B7WimeZac23zhg,1dV3qNEv8nNUAX1k3qdE2w,YHCseOJ93wJh0gBcii_2qA,TJt1W9haRm2DKuoZLQ69yA,wM8QNs7uSyDqMJKjBYFPCQ,gp_bu7Ah81qaBY3M0Leffw,PUKOr5bEI87TVHjwijT1xw,zV38gkkEeJ4cVRlSWWQTfQ,H1j34TgbrVZkxeww9xlJTw,F8M0IukXQqR50IRyocRQbg
UgMW8bLE0QMJDCkQ1Ax5Mg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aq_ZxGHiri48TUXJlpRkCQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
jOERvhmK6_lo_XGUBPws_w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2hRe26HSCAWbFRn5WChK-Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0j21z2Q1HGic7jW6e9h7A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Content-Based Recommender

In [200]:
def nameify(name):
    return name.replace('&', 'and').replace(' ', '_').lower()

def build_business_profiles(business: pd.DataFrame) -> pd.DataFrame:
    # TODO: Feature engineering (10 points)

    business = business.set_index('business_id')
    
    # I'll use the categories, which I've explored in my EDA above. I think they're a pretty good fit.
    features = pd.DataFrame(0, index=business.index, columns=map(nameify, category_counts.index))
    for bid in features.index:
        for cat in business.loc[bid, 'categories'].split(', '):
            if nameify(cat) in features.columns:
                features.loc[bid, nameify(cat)] = 1
    
    # I'll also add the location, since recommending based on proximity seems like a decent idea.
    # I normalized them to the 0-1 range, since otherwise they would be wildly out of scale with the one-hot features
    # features['latitude'] = (business.latitude - business.latitude.min()) / (business.latitude.max() - business.latitude.min())
    # features['longitude'] = (business.longitude - business.longitude.min()) / (business.longitude.max() - business.longitude.min())
    # UPDATE: turns out this doesn't work when calculating user profiles later, they just cancel out and don't work as intended
    return features

df_business_profiles = build_business_profiles(df_yelp_business)
df_business_profiles.head()

,ethnic_food,food_trucks,specialty_food,imported_food,argentine,food,restaurants,grocery,middle_eastern,japanese,...,mags,music_and_video,hungarian,new_mexican_cuisine,seafood_markets,whiskey_bars,day_spas,health_and_medical,bed_and_breakfast,fitness_and_instruction
business_id,,,,,,,,,,,,,,,,,,,,,
pQeaRpvuhoEqudo3uymHIQ,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CsLQLiRoafpJPJSkNX2h5Q,0,0,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
lu7vtrp_bE9PnxWfA8g4Pg,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
vjTVxnsQEZ34XjYNS-XUpA,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fnZrZlqW1Z8iWgTVDfv_MA,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
def build_user_profiles(utility_matrix: pd.DataFrame, 
                                   business_profiles: pd.DataFrame) -> pd.DataFrame:
    # TODO: Feature aggregation (5 points)
    features = pd.DataFrame(0, index=utility_matrix.index, columns=business_profiles.columns).astype(np.float64)
    
    counts = {}
    for uidx, bidx in zip(*utility_matrix.to_numpy().nonzero()):
        features.iloc[uidx] += business_profiles.iloc[bidx] * utility_matrix.iloc[uidx, bidx]
        counts[uidx] = counts.get(uidx, 0) + 1

    for uidx, cnt in counts.items():
        features.iloc[uidx] /= cnt
        features.iloc[uidx] /= cnt

    return features

df_user_profiles = build_user_profiles(df_utility_matrix, df_business_profiles)
df_user_profiles.head()

,ethnic_food,food_trucks,specialty_food,imported_food,argentine,food,restaurants,grocery,middle_eastern,japanese,...,mags,music_and_video,hungarian,new_mexican_cuisine,seafood_markets,whiskey_bars,day_spas,health_and_medical,bed_and_breakfast,fitness_and_instruction
UgMW8bLE0QMJDCkQ1Ax5Mg,0.00,0.0,0.00,0.0,0.0,0.062500,0.187500,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aq_ZxGHiri48TUXJlpRkCQ,0.00,0.0,0.00,0.0,0.0,0.000000,0.666667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
jOERvhmK6_lo_XGUBPws_w,0.25,0.0,0.25,0.0,0.0,0.375000,0.625000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2hRe26HSCAWbFRn5WChK-Q,0.00,0.0,0.00,0.0,0.0,0.000000,0.500000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0j21z2Q1HGic7jW6e9h7A,0.00,0.0,0.00,0.0,0.0,0.090909,0.500000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [205]:
df_user_profiles.describe()

,ethnic_food,food_trucks,specialty_food,imported_food,argentine,food,restaurants,grocery,middle_eastern,japanese,...,mags,music_and_video,hungarian,new_mexican_cuisine,seafood_markets,whiskey_bars,day_spas,health_and_medical,bed_and_breakfast,fitness_and_instruction
count,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,...,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000,16120.000000
mean,0.005732,0.003062,0.019801,0.001456,0.000192,0.134319,0.438342,0.004975,0.010089,0.042254,...,0.000143,0.000143,0.000244,0.003287,0.000863,0.000810,0.002368,0.000493,0.000020,0.000185
std,0.050824,0.035721,0.090798,0.028828,0.008312,0.243653,0.431887,0.042490,0.062974,0.141188,...,0.007337,0.007337,0.010354,0.036331,0.024965,0.018802,0.032175,0.014450,0.004766,0.009981
min,-1.000000,-0.500000,-1.000000,-0.500000,-0.500000,-1.000000,-1.000000,-0.500000,-0.500000,-1.000000,...,-0.333333,-0.333333,-0.166667,-0.500000,-1.000000,-0.500000,-0.500000,-0.500000,-0.250000,-0.333333
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.187500,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.750000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,0.333333,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.333333,0.333333,0.500000,1.000000,1.000000,0.500000,1.000000,1.000000,0.250000,0.333333


In [188]:
def euclidian_sim(a, b):
    dist = np.sqrt((a - b).pow(2).sum())
    return 1 / (1 + dist)

def predict_content_ratings(user_profiles: pd.DataFrame, business_profiles: pd.DataFrame) -> pd.DataFrame:
    print(euclidian_sim(user_profiles.iloc[3], business_profiles.iloc[2]))
    # TODO: Distance based rating prediction (5 points)
    ratings = pd.DataFrame(0, index=user_profiles.index, columns=business_profiles.index,dtype=np.float64)
    return ratings
    
    for uid in user_profiles.index[:5]:
        user = user_profiles.loc[uid]
        business_sim = business_profiles - user
        
        # Now take difference with 
        euclid = business_sim.abs().sum(axis=1) #Manhetten distance
        
        mini = business_df.iloc[business_df.argmin()]
        maxi = business_df.iloc[business_df.argmax()] - mini
        business_df-=mini
        business_df/=maxi
        business_df = 1 - business_df

        df.loc[user] = business_df
    
    return ratings

df_content_predictions = predict_content_ratings(df_user_profiles, df_business_profiles)
df_content_predictions.head()

1.0817301915403086


business_id,pQeaRpvuhoEqudo3uymHIQ,CsLQLiRoafpJPJSkNX2h5Q,lu7vtrp_bE9PnxWfA8g4Pg,vjTVxnsQEZ34XjYNS-XUpA,fnZrZlqW1Z8iWgTVDfv_MA,rVBPQdeayMYht4Uv_FOLHg,fhNf_sg-XzZ3e7HEVGuOZg,LoRef3ChgZKbxUio-sHgQg,Ga2Bt7xfqoggTypWD5VpoQ,xFc50drSPxXkcLvX5ygqrg,...,2SfSzEd3B7WimeZac23zhg,1dV3qNEv8nNUAX1k3qdE2w,YHCseOJ93wJh0gBcii_2qA,TJt1W9haRm2DKuoZLQ69yA,wM8QNs7uSyDqMJKjBYFPCQ,gp_bu7Ah81qaBY3M0Leffw,PUKOr5bEI87TVHjwijT1xw,zV38gkkEeJ4cVRlSWWQTfQ,H1j34TgbrVZkxeww9xlJTw,F8M0IukXQqR50IRyocRQbg
UgMW8bLE0QMJDCkQ1Ax5Mg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aq_ZxGHiri48TUXJlpRkCQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
jOERvhmK6_lo_XGUBPws_w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2hRe26HSCAWbFRn5WChK-Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0j21z2Q1HGic7jW6e9h7A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Collaborative Filtering Recommender

In [ ]:
def predict_collaborative_ratings(utility_matrix: pd.DataFrame) -> pd.DataFrame:
    # TODO: User-item collaborative filtering based rating prediction (15 points)
    # TODO: UV-decomposition based rating prediction (optional for 10 extra points)
    pass

df_collaborative_predictions = predict_collaborative_ratings(df_utility_matrix)

## Evaluation

In [ ]:
def score_model(utility_matrix: pd.DataFrame, predicted_utility_matrix: pd.DataFrame, model_name="model_0"):
    # TODO: Implement these by hand (each metric 1 point)
    rmse_score = 0
    map_score = 0
    coverage_score = 0
    personalization_score = 0
    intra_list_similarity_score = 0
    
    print("{} RMSE {}".format(model_name, rmse_score))
    print("MAP: {}".format(model_name, map_score))
    print("Coverage: {}".format(model_name, coverage_score))
    print("Personalization: {}".format(model_name, personalization_score))
    print("Intra-list similarity: {}".format(model_name, intra_list_similarity_score))    

score_model(df_content_predictions, df_utility_matrix, "content-based approach")
score_model(df_collaborative_predictions, df_utility_matrix, "collaborative-filtering approach")

## Constraint Satisfaction Problem

We can work with the task of planing breakfast, lunch and dinner for particular user as Constraint Satisfaction Problem with

**Domain**: {all_businesses}

**Variables**: {breakfast, lunch, dinner}

**Constraints**: {constrainst regarding individual variable, or several variables at once}

We also have predicted ratings for every business and want to have personalized plan of restaurants. So we won't only satisfy our constraints, but also would like to get the maximum cumulative rating.

Take a look on prepared constraints and finish empty constraints in similar way (some of these constraints may require analytics on business data. e.g. to finish **has_coffee_constraint** you may need to determine all the categories which may include good coffee in their menu)

In [ ]:
def is_vegetarian_constraint(business_id):
    return "vegetarian" in df_yelp_business[df_yelp_business["business_id"] == business_id].categories.values[0].lower()

def has_coffee_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return False

def has_alcohol_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_at_date_at_time_meta_constraint(weekday, time, business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_at_monday_at_10am_constraint(business_id):
    return is_open_at_date_at_time("monday", "10:00", business_id)

def all_are_different_constraint(state):
    for time in ["breakfast", "dinner", "lunch"]:
        for _t in ["breakfast", "dinner", "lunch"]:
            if time == _t: continue
            business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[time]["business_id"]].categories.values[0].split(","))
            _business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[_t]["business_id"]].categories.values[0].split(","))
            if len(business_categories.intersection(_business_categories)) > \
                    len(business_categories.union(_business_categories)) // 2:
                return False
    return True

def all_are_in_the_same_city_constraint(state):
    # TODO: implement this constraint (1 point)
    return False

def all_are_in_the_same_region_meta_constraint(coordinates, threshold, state):
    # TODO: implement this constraint (1 point). Hint: use haversine distance https://pypi.org/project/haversine/
    return False

def all_are_in_test_region(state):
    return all_are_in_the_same_region_constraint({"lat": 40.110446, "lon": -115.301568}, 400, state)

def at_least_one_visited_place_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one familiar place
    return False

def at_least_one_has_coffee_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one place with coffee
    return False

In [ ]:
import random 

random.seed(42)
inspected_user = random.choice(df_yelp_reviews["user_id"].unique())

all_constraints = {
    "breakfast": [has_coffee_constraint, is_open_constraint, is_open_at_monday_at_10am_constraint],
    "lunch": [is_open_constraint],
    "dinner": [is_vegetarian_constraint, has_alcohol_constraint, is_open_constraint],
    "state": [at_least_one_has_coffee_constraint, at_least_one_visited_place_constraint, all_are_in_test_region,
             all_are_in_the_same_city_constraint, all_are_different_constraint]
}

def goal_test(state: dict, constraints: dict):
    cumulative_rating = state["breakfast"]["predicted_rating"]*state["lunch"]["predicted_rating"]*\
                        state["dinner"]["predicted_rating"]
    for k in constraints.keys():
        if k == "state":
            for c in constraints[k]:
                cumulative_rating *= c(state)
        else:
            for c in constraints[k]:
                cumulative_rating *= c(state[k]["business_id"])
    return cumulative_rating


def prepare_restaurants_plan(user_id: str, user_business_ratings: pd.DataFrame, constraints: dict):
    # TODO: assign breakfast, lunch and dinner by solving Constraint Satisfaction Problem 
    # maximizing total score and satisfying all the constraints (it should work with any configuration of constraints)
    
    # You can implement Backtracking (10) + Filtering (10) + Ordering (5) using goal_test
    # OR
    # Local Search (10) + Min-Conflicts heuristic (10) + Ordering (5) with modification of goal test to work as Min-Conflicts heuristic
    
    state = {"user_id" : user_id,
        "breakfast": 
                {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0},
            "lunch": 
    {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0},
            "dinner": {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0}}
    
    state_v = goal_test(state, constraints)

    
    return state

# TODO: replace df_utility_matrix with your best predictions
prepare_restaurants_plan(inspected_user, df_utility_matrix, all_constraints)